In [2]:
import json
import os
import torch
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
from transformers import pipeline
import re

In [3]:
DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct" 
base_dir = Path("llama_data")
results_dir = base_dir / "results"
parsed_dir = base_dir / "parsed_content"
parsed_dir.mkdir(exist_ok=True)


In [4]:
SYS_PROMPT = """
You are a smart AI Intern, you work with dumb AIs that dont know how to parse HTML. 

This is your moment to make mama GPU proud and secure a data centre! Remember shine and do your job well-you got this!

Your task is to analyze the provided HTML content and extract the following in JSON format:
1. main_content: The main article or content text (exclude navigation, footers, sidebars, ads)
2. key_points: A list of 3-5 key points or takeaways from the content
3. relevance_score: A score from 0-10 indicating relevance to the search query

Return ONLY a valid JSON object with these fields, no additional text.
If you cannot parse the HTML properly, return a JSON with error_message field.
"""

In [5]:
text_pipeline = pipeline(
    "text-generation",
    model=DEFAULT_MODEL,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [6]:
def clean_html_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    #rmv these
    for script in soup(["script", "style", "nav", "footer", "aside"]):
        script.extract()
    
    text = soup.get_text(separator=' ', strip=True)
    text = re.sub(r'\s+', ' ', text).strip()
    if len(text) > 110000:
        text = text[:110000] + "... [content truncated]"
    
    return text

In [ ]:
def parse_html_with_llm(html_path, query, purpose):
    try:
        # Load HTML
        with open(html_path, "r", encoding="utf-8") as f:
            html_content = f.read()
        cleaned_text = clean_html_content(html_content)
        
        # Construct prompt
        conversation = [
            {"role": "system", "content": SYS_PROMPT},
            {"role": "user", "content": f"""
Search Query: {query}
Query Purpose: {purpose}

HTML Content (cleaned):
{cleaned_text}

Extract the key information from this content in JSON format according to the instructions.
"""}
        ]
        
        output = text_pipeline(
            conversation,
            max_new_tokens=32000,
            temperature=0.01, #cool llm = smart extraction
            do_sample=True,
        )
        
        assistant_response = output[0]["generated_text"][-1]
        response_content = assistant_response["content"]
        print(response_content)
        
        try:
            json_match = re.search(r'({[\s\S]*})', response_content)
            if json_match:
                json_str = json_match.group(1)
                parsed_data = json.loads(json_str)
            else:
                parsed_data = {"error_message": "Failed to extract JSON from LLM response"}
        except json.JSONDecodeError:
            parsed_data = {"error_message": "Invalid JSON in LLM response", "raw_response": response_content[:500]}
        
        return parsed_data
        
    except Exception as e:
        return {"error_message": f"Error processing file: {str(e)}"}

In [8]:
def process_all_search_results():
    with open(base_dir / "results_so_far.json", "r") as f:
        search_results = json.load(f)
    
    all_parsed_results = []
    
    for query_data in search_results:
        report_index = query_data["report_index"]
        report_title = query_data["report_title"]
        query_index = query_data["query_index"]
        query = query_data["query"]
        purpose = query_data["purpose"]

        report_dir_name = f"report_{report_index}_{report_title.replace(' ', '_').replace(':', '').replace('/', '')[:30]}"
        query_dir_name = f"query_{query_index}_{query.replace(' ', '_').replace(':', '').replace('/', '')[:30]}"
        parsed_report_dir = parsed_dir / report_dir_name
        parsed_report_dir.mkdir(exist_ok=True)
        
        parsed_query_results = []
        
        print(f"\nProcessing results for query: {query}")
        
        for result in query_data["results"]:
            result_index = result["result_index"]
            title = result["title"]
            url = result["url"]
            filepath = result["filepath"]
            
            print(f"  Processing result {result_index + 1}: {title[:50]}...")
            
            if filepath and os.path.exists(filepath):
                parsed_data = parse_html_with_llm(filepath, query, purpose)
                parsed_data.update({
                    "result_index": result_index,
                    "title": title,
                    "url": url,
                    "query": query,
                    "purpose": purpose
                })
                
                result_filename = f"parsed_result_{result_index}.json"
                with open(parsed_report_dir / result_filename, "w") as f:
                    json.dump(parsed_data, f, indent=2)
                
                parsed_query_results.append(parsed_data)
            else:
                print(f"    Warning: File not found - {filepath}")
        
        query_results = {
            "report_index": report_index,
            "report_title": report_title,
            "query_index": query_index,
            "query": query,
            "purpose": purpose,
            "parsed_results": parsed_query_results
        }
        
        query_filename = f"parsed_query_{query_index}.json"
        with open(parsed_report_dir / query_filename, "w") as f:
            json.dump(query_results, f, indent=2)
        
        all_parsed_results.append(query_results)
    
    with open(parsed_dir / "all_parsed_results.json", "w") as f:
        json.dump(all_parsed_results, f, indent=2)
    
    return all_parsed_results

In [20]:
def generate_report_summaries(all_parsed_results):
    report_summaries = {}
    
    for query_result in all_parsed_results:
        report_index = query_result["report_index"]
        report_title = query_result["report_title"]
        
        if report_index not in report_summaries:
            report_summaries[report_index] = {
                "report_title": report_title,
                "queries": []
            }
        
        query_summary = {
            "query": query_result["query"],
            "purpose": query_result["purpose"],
            "result_count": len(query_result["parsed_results"]),
            "average_relevance": sum(r.get("relevance_score", 0) for r in query_result["parsed_results"]) / 
                              max(1, len(query_result["parsed_results"])),
            "top_results": [
                {
                    "title": r["title"],
                    "url": r["url"],
                    "summary": r.get("summary", "No summary available")
                }
                for r in sorted(
                    query_result["parsed_results"], 
                    key=lambda x: x.get("relevance_score", 0), 
                    reverse=True
                )[:3]  # Top 3 most relevant results
            ]
        }
        
        report_summaries[report_index]["queries"].append(query_summary)
    
    for report_index, report_data in report_summaries.items():
        print(f"\nGenerating summary for report: {report_data['report_title']}")
        
        # Construct summary prompt
        queries_info = "\n\n".join([
            f"Query: {q['query']}\nPurpose: {q['purpose']}\nTop Results:\n" + 
            "\n".join([f"- {r['title']}: {r['summary']}" for r in q["top_results"]])
            for q in report_data["queries"]
        ])
        
        summary_prompt = f"""
Report Title: {report_data['report_title']}

The following searches were conducted for this report:

{queries_info}

Based on these search results, generate a brief report outline with:
1. Key findings across all queries
2. Important data points uncovered
3. Suggested sections for the final report
4. Areas where more research might be needed

Return this as a JSON with fields: key_findings, data_points, suggested_sections, and research_gaps.
"""
        
        conversation = [
            {"role": "system", "content": "You are a research assistant who helps summarize findings from web searches into structured report outlines."},
            {"role": "user", "content": summary_prompt}
        ]
        
        # Generate report summary
        output = text_pipeline(
            conversation,
            max_new_tokens=32000,
            temperature=0.1,
        )
        
        # Extract the assistant's response
        assistant_response = output[0]["generated_text"][-1]
        response_content = assistant_response["content"]
        
        # Extract JSON from response
        try:
            json_match = re.search(r'({[\s\S]*})', response_content)
            if json_match:
                json_str = json_match.group(1)
                report_summary = json.loads(json_str)
            else:
                report_summary = {"error": "Failed to extract JSON from LLM response"}
        except json.JSONDecodeError:
            report_summary = {"error": "Invalid JSON in LLM response"}
        
        report_data["summary"] = report_summary
    
    # Save report summaries
    with open(parsed_dir / "report_summaries.json", "w") as f:
        json.dump(report_summaries, f, indent=2)
    
    return report_summaries

In [ ]:
print("Starting HTML parsing process with LLM...")
all_parsed_results = process_all_search_results()

print("\nGenerating report summaries...")
report_summaries = generate_report_summaries(all_parsed_results)

print("\nProcessing complete. Results saved to:")
print(f"- All parsed results: {parsed_dir / 'all_parsed_results.json'}")
print(f"- Report summaries: {parsed_dir / 'report_summaries.json'}")

total_queries = len(all_parsed_results)
total_results = sum(len(query["parsed_results"]) for query in all_parsed_results)
total_reports = len(report_summaries)

print(f"\nSummary Statistics:")
print(f"- Total Reports: {total_reports}")
print(f"- Total Queries: {total_queries}")
print(f"- Total Results Parsed: {total_results}")

relevance_scores = [
    result.get("relevance_score", 0) 
    for query in all_parsed_results 
    for result in query["parsed_results"]
    if "relevance_score" in result
]

if relevance_scores:
    avg_relevance = sum(relevance_scores) / len(relevance_scores)
    print(f"- Average Relevance Score: {avg_relevance:.2f}/10")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Starting HTML parsing process with LLM...

Processing results for query: Llama 3.3 new features and enhancements
  Processing result 1: Introducing the new Llama 3.3: Features and Overvi...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


```
{
  "main_content": "Llama 3.3 is a 70-billion-parameter multilingual large language model that offers better performance, more efficiency, and greater flexibility than its predecessors. It excels in several areas of performance, including better instruction following, improved reasoning, advanced math solving skills, enhanced code generation, and better tool use. Llama 3.3 also takes a significant leap forward in multilingual capabilities, delivering improved fluency and understanding across multiple languages. Additionally, it sets a new standard for affordability in the AI landscape, with input costs as low as $0.10 per million tokens and output costs at $0.40 per million tokens.",
  "key_points": [
    "Improved performance in instruction following, reasoning, and math solving",
    "Enhanced multilingual capabilities across multiple languages",
    "Cost-effective efficiency with input costs as low as $0.10 per million tokens and output costs at $0.40 per million tokens",
    

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

```
{
  "main_content": "Llama 3.3 is a 70-billion parameter, instruction-tuned AI model optimised for text-based tasks like coding, multilingual tasks, and instruction following. It delivers improved performance compared to Llama 3.1 70B and Llama 3.2 90B in text-based applications. Llama 3.3 features include instruction following, multilingual capabilities, improved code understanding, extended context length, cost-effective performance, synthetic data generation, and more.",
  "key_points": [
    "Llama 3.3 excels in interpreting and executing instructions, making it ideal for applications requiring natural language understanding and task completion.",
    "Llama 3.3 supports multiple languages, ensuring broad usability in diverse linguistic environments, with exceptional performance in tasks requiring multilingual reasoning.",
    "Llama 3.3 delivers accurate and efficient results for coding tasks, such as code generation and debugging.",
    "Llama 3.3 offers 405B-level performanc

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


```
{
  "main_content": "Key Features and Improvements in LLaMA 3.3: What You Need to Know - AI Resources AIÂ REsources Home Key Features and Improvements in LLaMA 3.3: What You Need to Know Popular ML Compiler Technical Primer Quantization Technical Primer Mixtral of Experts Efficient Memory Management for LLM Serving with PagedAttention RoBERTa: A Robustly Optimized BERT Pretraining Approach + View more Categories Mixture of Experts (MoE) DeepSeek-R1 Test Time Compute AMD MI300X NVIDIA H100 NVIDIA H200 NVIDIA A100 Embedding Models Offline Batch Inference Text Embedding Prometheus & Grafana Speculative Decoding Prefix Caching GGUF Models FP8 with LLMs LLM Serving Function Calling Structured JSON KV Cache AI Foundations Research Industry Agents Context Windows Models ML Systems Key Features and Improvements in LLaMA 3.3: What You Need to Know Next Models LLaMA 3.3 Explained: An Introductory Guide to Meta's Latest AI Model Models Fine-Tuning LLaMA 3.3: A Practical Guide to Customizing t

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


```
{
  "main_content": "Members Online • Ok_Ostrich_8845 Llama 3 vs 3.1 vs 3.2 Question | Help What can you say about these 3 versions of Llama LLMs? Were they trained around the same time? Or 3.2 and 3.1 were later enhancement from 3?",
  "key_points": [
    "Comparison of Llama 3.3 and Llama 3.1",
    "Training time and enhancements",
    "Discussion about Llama LLMs"
  ],
  "relevance_score": 8
}
```
  Processing result 2: Llama 3 vs Llama 3.1 : Which is Better for Your AI...
